<a href="https://colab.research.google.com/github/allakoala/data_science/blob/main/colab_notebooks/HomeWork_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#HW: https://docs.google.com/document/d/1l6R-wlxfCrL_KBRKUmHvOjBRShzPyzDf0xAbGOD0cGs/edit
# **Part 1: EDA**

#сonclusions:
1. there are categorical features with "unknown" values: job, marital, education, default, housing, loan.
2. for the following features the NAN values could be dropped because those columns could be dependable:job, marital, education
3. the following features should be replaced by median or mode: default, housing, loan

#assumptions (should be discussed with SME):
1. the following columns could be dependable and should be handled (dropped or groupped):job, marital, education
2. the next substitution for the job feature could be made:
if education = 'unknown' and 16<age<21, then job = student

In [ ]:
#import libriaries
#basics
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings(action = 'ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import DBSCAN
import math

#data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#path of the file to read
url = "/content/drive/MyDrive/data/bank-additional-full.csv"

#read the file into a variable
data = pd.read_csv(url, sep=';')

#examine the data for Univariate analysis: consider features separately, their distribution, descriptive statistics, anomalies, omissions, etc
data.head()

In [ ]:
data.tail()

In [ ]:
#devide features into num and categirical

#empty lists for numerical and categorical features
numerical_cols = []
categorical_cols = []

#loop over each column and determine its data type
for col in data.columns:
    if data[col].dtype == 'object':
        categorical_cols.append(col)
    else:
        numerical_cols.append(col)

print("Numerical features:", numerical_cols)
print("Categorical features:", categorical_cols)

In [ ]:
#find duplicate rows
duplicate_rows = data.duplicated(subset=numerical_cols+categorical_cols, keep="first")
duplicate_rows.sum()

In [ ]:
#remove duplicate rows
#data = data[~duplicate_rows]
data = data.drop_duplicates()

In [ ]:
#identify the data type
data.info()

In [ ]:
#for each dataset column print unique values
for col in data.columns:
    n_unique_values = data[col].nunique()
    unique_values = data[col].unique()
    print(f"{col}: {n_unique_values}: {unique_values}")

In [ ]:
#look at the "unknown" value count in categorical features
data_copy = data.copy()
data_copy[categorical_cols] = data_copy[categorical_cols].replace('unknown', np.nan)
data_copy.isna().sum()

#conclusions:
1. the dataset in terms of the target lable is Imbalanced, which means  that for 'y' need to downsample the majority class and upweight the downsampled class.
2. this is a binary classification problem

In [ ]:
#target lable is a categorical value
target = data['y']
#check the distribution: descriptive summary statistics and visualization tools to check the distribution of the variable(s)
#frequency table using value_counts()
target.value_counts()

In [ ]:
target.describe()

#conclusions:
1. mean < median (distribution of values in that column is negatively skewed and it has a long tail on the left-hand side of the median and is more spread out towards the lower values. there are more observations with lower values than with higher values): pdays, emp.var.rate, cons.price.index, euribor3m, nr.employed
2. mean > median (distribution of values in that column is positevly skewed, presence of abnormal high values): age, duration, campaign, previous, cons.conf.idx
3. big difference between 75th %tile and max values (sign of extreme values-Outliers, check out box plots): age, duration, campaign, previous, cons.conf.idx
4. high standard deviation (values are more spread out and have a wider range of values) - age, duration, pdays, nr.employed
5. low standard deviation (values are more tightly clustered around the mean and have a narrower range of values) - campaign, previous, emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m
6. positively correlated pairs (positive slope):
  1. age v duration
  2. age v campaign
7. negative correlation: duration and campaign

#assumptions
1. Age histogram shows that much less people participate in marketing campaigns after 60
2. the most active age is between 30 - 50 years
2. Age Min value is 17 - is this eligible? This entry should be deleted

In [ ]:
#continuous variables distribution visualization using a histogram

#summary statistics of the numerical features
print(data[numerical_cols].describe())

#the histogram is colored by the target variable 'y' (which is binary) to see the differences in the distribution between the two target classes.
for col in numerical_cols:
    sns.histplot(data=data, x=col, hue='y', kde=True)
    plt.show()

In [ ]:
#scatterplot method
sns.set()
sns.pairplot(data[numerical_cols], size = 2.5)
plt.show();

In [ ]:
#scatter plot for each pair of numerical columns
outcome = 'y'

for i in range(len(numerical_cols)):
    for j in range(i+1, len(numerical_cols)):
      #create scatter plot with color-coded points
      sns.scatterplot(x=numerical_cols[i], y=numerical_cols[j], hue=outcome, data=data)

      #add regression line
      sns.regplot(x=numerical_cols[i], y=numerical_cols[j], data=data, scatter=False, color="black")

      #set plot title and axis labels
      plt.title(f"{numerical_cols[i]} vs {numerical_cols[j]}")
      plt.xlabel(numerical_cols[i])
      plt.ylabel(numerical_cols[j])

      #display plot
      plt.show()

#conclusions:
1. people with housing loan and without personal loan are the most active users
2. peopel tend to ignore campaigns in december
3. admins are the most active category

In [ ]:
#categorical values analysis

#frequency table of the categorical features
for col in categorical_cols:
  print(data[col].value_counts())

#distribution of the categorical features visualisation using a bar plot. The bar plot is colored by the target variable 'y' to see the differences in the distribution between the two target classes.
for col in categorical_cols:
  sns.countplot(data=data, x=col, hue='y')
  plt.xticks(rotation=90)
  plt.show()

#conclusions:
we don't need an imputation (fillna() or dropna())

In [ ]:
#missing data for each variable and way to handle it. missing data can imply a reduction of the sample size

total = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
total_2 = data.isna().sum().sort_values(ascending=False)
percent_2 = (data.isna().sum()/data.isna().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent, total_2, percent_2], axis=1, keys=['Tota_null', 'Percent_null', 'Total_na', 'Percent_na'])
missing_data

#conclusions:
1. this dataset comprises some outliers according to several methods of detection
2. the most Low range values are similar and not too far from 0
3. High range values are far from 0 and the following columns seem to have outliers: campaign, previous

In [ ]:
#outliers detection

#Interquartile Range (IQR) method (values outside the normal range)
for col in numerical_cols:
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = data[(data[col] < lower) | (data[col] > upper)]
    print(f"{col} has {len(outliers)} outliers")

In [ ]:
#boxplot
#red lines in the boxplot indicate the lower and upper limits of the normal range (calculated using the IQR method), and any points outside of these lines are considered outliers
for col in numerical_cols:
    fig, ax = plt.subplots()
    ax.boxplot(data[col])
    ax.set_title(f"{col} Distribution")
    ax.set_ylabel(col)
    ax.axhline(y=lower, color='r', linestyle='-', label='Lower Limit')
    ax.axhline(y=upper, color='g', linestyle='-', label='Upper Limit')
    ax.text(0.75, lower, f"{lower:.2f}", va='center', ha='center', bbox=dict(facecolor='red', alpha=0.5), fontsize=12)
    ax.text(0.75, upper, f"{upper:.2f}", va='center', ha='center', bbox=dict(facecolor='green', alpha=0.5), fontsize=12)
    ax.legend()
    plt.show()

In [ ]:
#standardizing data method for outliers
for col in numerical_cols:
    value_scaled = StandardScaler().fit_transform(data[col][:,np.newaxis])
    low_range = value_scaled[value_scaled[:, 0] < -(data[col].std())]
    high_range = value_scaled[value_scaled[:, 0] > data[col].std()]
    print(f"{col}: \nLow range: \n{low_range} \nHigh range: \n{high_range}\n")

In [ ]:
#anomalies or omissions

#one-dimentional dataset method
#define a list to accumlate anomalies
anomalies = []

#loop through each column
for col in data[numerical_cols].columns:
    #compute the mean and standard deviation of the column
    col_mean = data[col].mean()
    col_std = data[col].std()

    #define the upper and lower limits for outliers
    lower_limit = col_mean - 3 * col_std
    upper_limit = col_mean + 3 * col_std

    #find the outliers in the column and append them to the anomalies list
    col_anomalies = data[(data[col] < lower_limit) | (data[col] > upper_limit)][col].values.tolist()
    anomalies += col_anomalies

    #print the results for the column
    print(f"Column {col}: Lower Limit = {lower_limit:.2f}, Upper Limit = {upper_limit:.2f}, Anomalies = {col_anomalies}")

In [ ]:
#Isolation forest https://towardsdatascience.com/isolation-forest-auto-anomaly-detection-with-python-e7a8559d4562
from sklearn.ensemble import IsolationForest

#initialize the isolation forest model
model = IsolationForest(n_estimators=100, contamination='auto', random_state=42)

#loop through each column and fit the model to identify anomalies
for col in data.columns:
    #preprocess the data
    X = pd.get_dummies(data[col]) if col in categorical_cols else data[[col]]

    #fit the model and predict the anomalies
    model.fit(X)
    anomalies = model.predict(X) == -1

    #list of anomalies per column
    #print(f'Anomalies in column {col}: {data[anomalies][col].tolist()}')

    #list of anomalies unique values
    print(f'Unique anomalies in column {col}: {data[anomalies][col].unique().tolist()}')

In [ ]:
#DBScan Clustering - craches the whole page :(

# DBSCAN clustering parameters
#min_samples = 2
#eps = 3

# list to accumulate anomalies per column
#anomalies_list = []

# loop over each numerical column in the dataset
#for col in numerical_cols:
    # extract the column data as a numpy array
    #col_data = np.array(data[col]).reshape(-1, 1)

    # DBSCAN clustering on the column data
    #anomalies_detection = DBSCAN(min_samples=min_samples, eps=eps)
    #clusters = anomalies_detection.fit_predict(col_data)

    # anomalies == points assigned to -1 cluster label
    #anomalies = col_data[clusters == -1]
    #anomalies_list.append(anomalies)

    #print(f"Column {col}: Anomalies = {anomalies}")

#conclusions:
1. pairs where one of the variables is almost perfectly predictable from the other variables, and could lead to problems in a regression model:
  1. emp.var.rate - cons.price.idx;
  2. emp.var.rate - euribor3m;
  3. emp.var.rate - nr.employed
2. pairs of features which have a high positive correlation coefficient (close to 1), then they are likely measuring similar aspects of the data and may be redundant:
  1. cons.price.idx - emp.var.rate;
  2. euribor3m - emp.var.rate;
  3. nr.employed- emp.var.rate;
  4. cons.price.idx - euribor3m;
  5. cons.price.idx - nr.employed;
  6. euribor3m - nr.employed
3. pairs of features which have a high negative correlation coefficient (close to -1), then they may be measuring opposite aspects of the data and may be providing conflicting information:
  1. nr.employed - previous;
  2. previous - pdays

In [ ]:
#Multivariate analysis: consider features in pairs, try to introduce new features to consider their relationship with other features, etc.

#multicollinearity in Regression Analysis (correlation matrix)
corr_matrix = data.corr()
print(corr_matrix)

In [ ]:
#plot a correlation matrix plot
plt.matshow(corr_matrix)
plt.colorbar()
plt.show()

In [ ]:
#heatmap style
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

#Feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import ExtraTreesClassifier

categorical_cols = pd.Index(categorical_cols)
categorical_cols = categorical_cols.drop('y')
cat_data_fs = pd.get_dummies(data_copy[categorical_cols]) #categorical input data (One-hot encode categorical features)
num_data_fs = data_copy[numerical_cols] #numeric_input data
outcome_feature_y_fs = data_copy['y'].replace({'yes': 1, 'no': 0})
data_fs = pd.concat([cat_data_fs, num_data_fs, outcome_feature_y_fs], axis=1) #all input_data with NaN
data_fs = data_fs.fillna(data_fs.mode().iloc[0]) #substitute na with mode

print(data_fs)

In [ ]:
#select numerical features with highest correlation
num_k_best = SelectKBest(lambda X, Y: np.array(list(map(lambda x: np.abs(np.corrcoef(x, Y)[0, 1]), X.T))).T, k=1).fit_transform(data_fs[numerical_cols], data_fs['y'])

#train extra trees classifier to estimate feature importance for categorical features
et = ExtraTreesClassifier(random_state=42)
et.fit(data_fs.drop('y', axis=1), data_fs['y'])
categorical_importance = et.feature_importances_

#select categorical features with highest importance
cat_k_best = SelectKBest(lambda X, Y: np.array(list(map(lambda x: abs(np.corrcoef(x, Y)[0, 1]), X.T))).T, k=1).fit_transform(data_fs.drop(['y']+numerical_cols, axis=1), data_fs['y'])

#concatenate selected features with binary feature
selected_features = np.concatenate((num_k_best, cat_k_best, data_fs['y'].values.reshape(-1, 1)), axis=1)

#save selected features to file
pd.DataFrame(selected_features).to_csv("mydataset_selected.csv", index=False)

# **Part 2: DP**

#Data cleansing

In [ ]:
#load the data with the list of feature names
feature_names = ['age', 'type_of_job', 'marital_st', 'education', 'credit_in_default', 'housing_loan', 'personal_loan', 'contact_type', 'last_contact_month', 'last_day_of_week', 'last_contact_duration', 'campaign_n', 'past_days', 'pr_n_of_contacts', 'pr_outcom', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'n_employees', 'subscribtion']
bm_data = pd.read_csv(url, sep=';', names=feature_names, header=0)
bm_data = bm_data.drop_duplicates() #remove duplicates

#data randomization
bm_data = bm_data.reindex(np.random.permutation(bm_data.index))
print("Data set loaded. Num examples: ", len(bm_data))

In [ ]:
#data groupping for categorical and numerical features, converting + cleaning
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder

LABEL_NAME = 'subscribtion'

numeric_feature_names = ['age', 'last_contact_duration', 'campaign_n', 'past_days', 'pr_n_of_contacts', 'emp_var_rate', 'cons_price_idx', 'cons_conf_idx', 'euribor3m', 'n_employees']
categorical_feature_y_names = list(set(feature_names) - set(numeric_feature_names))
categorical_feature_input_names = list(set(feature_names) - set(numeric_feature_names) - set([LABEL_NAME]))

#missing values - replace 'unknown' values in categorical features with NaN so that we may replace it with mode
bm_data[categorical_feature_y_names] = bm_data[categorical_feature_y_names].replace('unknown', np.nan)
#print(bm_data[categorical_feature_names])

#for the following features drop nan: job, marital, education
bm_data = bm_data.dropna(subset=['type_of_job', 'marital_st', 'education'])
print(bm_data)

In [ ]:
#convert categorical features to numeric using one-hot encoding and replace nan by mode (mode is a robust statistic, meaning it will not be greatly affected by outliers)
cat_data = pd.get_dummies(bm_data[categorical_feature_input_names]) #cat into num input data without 'subscribtion', cos there will be 2 outcome features
print(cat_data)

In [ ]:
num_data = bm_data[numeric_feature_names] #numeric_input data
X = pd.concat([cat_data, num_data], axis=1) #all input_data with NaN
X = X.fillna(X.mode()) #all input_data with mode instead nan

#convert object columns to numeric
for col in X.select_dtypes(include=['object']):
    try:
        X[col] = pd.to_numeric(X[col], downcast='unsigned')
    except ValueError:
        X[col] = X[col].astype('category').cat.codes
    except:
        print('Error converting column', col)
X = X.astype('uint8')

print (X)

In [ ]:
#check the distribution

#plot distribution
for col in X.columns:

    #calculate distribution statistics
    mean = X[col].mean()
    std_dev = X[col].std()
    skew = X[col].skew()
    kurtosis = X[col].kurtosis()

    #distribution plots
    fig, ax = plt.subplots()
    sns.distplot(X[col], ax=ax)
    ax.axvline(mean, color='r', linestyle='--', label='mean')
    ax.axvline(mean-std_dev, color='g', linestyle='--', label='std dev')
    ax.axvline(mean+std_dev, color='g', linestyle='--')
    ax.set_title(f'{col} Distribution')
    ax.legend()
    plt.show()

#Outliers
how to delete outliers from Y - ? because at the Logistic regression model stage  X and Y have different number of rows

In [ ]:
#prepare Y
Y = bm_data[LABEL_NAME]
Y.describe()

In [ ]:
#replace 'yes' with 1 and 'no' with 0
Y = Y.replace({'yes': 1, 'no': 0}).astype(int)

# X = X + Y(converted)
X = pd.concat([X, Y], axis=1) #all data for outliers

X.info()

In [ ]:
#downsample the majority class and upweight the minority class with SMOTE classifier
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

df_majority = X[X[LABEL_NAME]==0]
df_minority = X[X[LABEL_NAME]==1]

print(df_majority)
print(df_minority)

In [ ]:
#downsample majority class
df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)

#upsample minority class using SMOTE
XX = X.drop([LABEL_NAME], axis=1)
YY = X[LABEL_NAME]

smote = SMOTE(random_state=42, k_neighbors=5)
XX_smote, YY_smote = smote.fit_resample(XX, YY)
df_minority_upsampled = pd.concat([XX_smote, YY_smote], axis=1)

#combine minority and downsampled majority classes
df_resampled = pd.concat([df_majority_downsampled, df_minority_upsampled])

#check class distribution
df_resampled[LABEL_NAME].value_counts()

In [ ]:
#outliers (Robust Z-score method - better for large datasets with more or less normal distribution) https://www.statology.org/remove-outliers-python/
#find absolute value of z-score for each observation
X = df_resampled
z = np.abs(stats.zscore(X))

#only keep rows in dataframe with all z-scores less than absolute value of 3
X_clean = X[(z<3).all(axis=1)]

print(X_clean)

#Preprocessing for data

In [ ]:
#data rescale with MaxScaler class
array = X_clean.values
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled = scaler.fit_transform(array)
#summarize transformed data
np.set_printoptions(precision=3)
print(rescaled)

In [ ]:
#Standatization
scaler_s = StandardScaler().fit(array)
rescaled_s = scaler_s.transform(array)
np.set_printoptions(precision=3)
print(rescaled_s)

In [ ]:
#Normalization
scaler_n = Normalizer().fit(array)
normalized = scaler.transform(array)
#summarize transformed data
np.set_printoptions(precision=3)
print(normalized)

In [ ]:
#Binarize data
binarizer = Binarizer(threshold=0.0).fit(array)
binary = binarizer.transform(array)
np.set_printoptions(precision=3)
print(binary)

In [ ]:
#check the distribution after Standatization
feature_names_all = X_clean.columns
rescaled_s_df = pd.DataFrame(rescaled_s, columns = feature_names_all)

In [ ]:
#plot distribution
for col in rescaled_s_df.columns:

    #calculate distribution statistics
    mean = rescaled_s_df[col].mean()
    std_dev = rescaled_s_df[col].std()
    skew = rescaled_s_df[col].skew()
    kurtosis = rescaled_s_df[col].kurtosis()

    #distribution plots
    fig, ax = plt.subplots()
    sns.distplot(X[col], ax=ax)
    ax.axvline(mean, color='r', linestyle='--', label='mean')
    ax.axvline(mean-std_dev, color='g', linestyle='--', label='std dev')
    ax.axvline(mean+std_dev, color='g', linestyle='--')
    ax.set_title(f'{col} Distribution')
    ax.legend()
    plt.show()

#Logistic regression model
after standartization Y is no binary?

In [ ]:
# import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# split data into train and test sets
X = rescaled_s_df.drop(LABEL_NAME, axis=1).astype(int) # input_data after upweighting
Y = rescaled_s_df[LABEL_NAME].astype(int) # outcome_feature_data after upweighting
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

# logistic regression model
model = LogisticRegression()
model.fit(X_train, Y_train)

# model evaluation
print('Accuracy on training set: ', model.score(X_train, Y_train))
print('Accuracy on test set: ', model.score(X_test, Y_test))

In [ ]:
#LGBMClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train, Y_train)
Y_pred = lgbm_clf.predict(X_test)

#model evaluation
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Accuracy: {accuracy}")

#model report
report = classification_report(Y_test, Y_pred)
print(report)

# **Part 3: DR**
#conclusions:
1.  Based on the scree plot, it appears that the first two components account for the majority of the variance in the data, while the remaining components account for a relatively small amount of additional variance. Therefore, the target number of components for this analysis could be set to 23.

In [ ]:
# Dimensionality Reduction Techniques

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD, PCA, NMF
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler

X = df_resampled.drop(LABEL_NAME, axis=1) #input_data after upweighting
y = df_resampled[LABEL_NAME] #outcome_feature_data after upweighting
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Singular Value Decomposition (SVD)
svd = TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X_scaled)

# SVD visualization
plt.scatter(X_svd[:,0], X_svd[:,1], c=y)
plt.title("SVD")
plt.show()

In [ ]:
# Principal Component Analysis (PCA)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# PCA visualization
plt.scatter(X_pca[:,0], X_pca[:,1], c=y)
plt.title("PCA")
plt.show()

In [ ]:
# Uniform Manifold Approximation and Projection (UMAP)
!pip install umap-learn
import umap

umap = umap.UMAP()
X_umap = umap.fit_transform(X_scaled)

# UMAP visualization https://umap-learn.readthedocs.io/en/latest/basic_usage.html
plt.scatter(X_umap[:,0], X_umap[:,1], c=y)
plt.title("UMAP")
plt.show()

In [ ]:
# t-SNE - doesnt work :(
#tsne = TSNE()
#X_tsne = tsne.fit_transform(X_scaled)
#fashion_scatter(X_tsne, y)

In [ ]:
# Non-negative Matrix Factorization (NMF) - doesnt work :(
#nmf = NMF()
#X_nmf = nmf.fit_transform(X_scaled)

In [ ]:
# Explained Variance

# SVD explained variance
svd_var = np.var(X_svd, axis=0)
svd_explained_var_ratio = svd_var / np.sum(svd_var)
print("SVD explained variance ratio:", svd_explained_var_ratio)

# PCA explained variance
print("PCA explained variance ratio:", pca.explained_variance_ratio_)

In [ ]:
# Selecting the best number of components
# PCA - Scree plot - №1 https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/
pca = PCA().fit(X_scaled)

plt.rcParams["figure.figsize"] = (12,6)

fig, ax = plt.subplots()
y = np.cumsum(pca.explained_variance_ratio_)
xi = np.arange(1, len(y)+1)

plt.ylim(0.0,1.1)
plt.xlim(0.0,30.5)
plt.plot(xi, y, marker='o', linestyle='--', color='b', label='Cumulative Variance')

plt.xlabel('Number of Components', fontsize=14)
plt.xticks(np.arange(1, 25, step=1)) #change from 0-based array index to 1-based human-readable label
plt.ylabel('Cumulative variance (%)', fontsize=14)
plt.title('The number of components needed to explain variance', fontsize=16)

plt.axhline(y=0.95, color='r', linestyle='-', label='95% Cut-Off Threshold')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=12)

#intersection point
cutoff_index = np.argmax(y > 0.95)
cutoff_x = xi[cutoff_index]
cutoff_y = y[cutoff_index]
plt.plot(cutoff_x, cutoff_y, marker='o', markersize=10, color='r', label='Intersection Point')


ax.grid(axis='x')
plt.legend(fontsize=12)
plt.show()

In [ ]:
# Selecting the best number of components

# PCA - Scree plot№2
pca = PCA()
pca.fit(X_scaled)

plt.plot(range(1,len(pca.explained_variance_ratio_)+1), pca.explained_variance_ratio_)
plt.title("PCA Scree plot")
plt.xlabel("Number of components")
plt.ylabel("Explained Variance Ratio")

#target number of components at 95% cut-off threshold
cumulative_var = np.cumsum(pca.explained_variance_ratio_)
target_components = np.argmax(cumulative_var >= 0.95) + 1

plt.axvline(x=target_components, color='r', linestyle='--', label='Target Number of Components')

plt.legend(fontsize=12)
plt.show()

**Mentor's Feedback**: In general, all acceptance criteria are met. Code is mostly well-structured, although in part 1 the header structure is more mixed up. Good and comprehensive conclusions are provided, it is clearly seen that the topic under research is well understood. **What to improve**: it is better to achieve more reusability of code. Imagine that you will do exactly the same sequence of actions not only on one dataset, but on 100 different datasets. In this case, it would be better if you implemented actions you would have to repeat as functions.